<div align="right">Бронников Егор ПМ-1901</div>

<h1 align="center">Индивидуальное задание: Построение адаптивных моделей</h1>

<div class="alert alert-warning">

<h3> Домашнее задание</h3>    
    
<p></p> 
Выполнять в отдельном файле!
    
<ul>1. Для <b>своего</b> ряда (см. папку Данные на de.unecon) определите наилучшую адаптивную модель прогнозирования. Для этого загрузите данные и отобразите их на графике. Есть ли в ряде тренд, сезонность?</ul>
    
<ul>2. С помощью библиотеки statsmodels обучите выбранную модель и постройте прогноз. Отобразите результат на графике.</ul>
    
<ul>3. Вычислите среднеквадратичную ошибку для оценки качества аппроксимации.</ul>
<p></p>
  
</div>

### Импорт модулей

In [ ]:
import pandas as pd
import numpy as np

from statsmodels.graphics.tsaplots import plot_acf

import matplotlib.pyplot as plt
%matplotlib inline

### Данные

In [ ]:
df = pd.read_csv("data/cfe-specialty-writing-papers-mon.csv", index_col="Month", parse_dates=True)
df = df.rename(columns={"CFE specialty writing papers monthly sales": "CFE"})

In [ ]:
df.head()

В связи с тем, что колонка с месяцем получилось довольной странного формата, её нужно преобразовать в удобный формат.

Допустим, что рассматриваются 2000-е.

In [ ]:
def convert_date(date: str):
    year, month = map(int, date.split("-"))
    year += 2000
    return pd.to_datetime("-".join(str(elem) for elem in [year, month]))

In [ ]:
df = df["CFE"]
df.index = pd.DatetimeIndex(map(convert_date , df.index), freq="MS")

In [ ]:
df.head()

In [ ]:
df.plot(figsize=(12,4))
plt.title("CFE specialty writing papers monthly sales")
plt.show()

**Коррелограмма**

In [ ]:
fig, ax = plt.subplots(figsize=(12,4), dpi=90)
plot_acf(df, lags=40, ax=ax)
plt.show()

### Выводы

Определённо есть сезонность каждые 12 месяцев, с трендом сложнее. Вроде бы явного тренда не наблюдается.

### Модели

Так как у нас есть сезонность, но нет тренда, то воспользуемся моделью Хольта-Уинтерса и просто в качестве аргумента `trend` зададим `None`.

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tools.eval_measures import mse

In [ ]:
holt_winters = ExponentialSmoothing(df, initialization_method="estimated",
                                    trend=None, seasonal="add").fit()

In [ ]:
holt_winters.model.params

In [ ]:
plt.figure(figsize=(12,4), dpi=150)
df.plot(label="CFE")
holt_winters.fittedvalues.plot(label="Holt-Winters")
plt.legend(loc="upper left")
plt.show()

**Среднеквадратичная ошибка**

In [ ]:
mse(df, holt_winters.fittedvalues)

По графику можно видеть, что наша аппроксимация далека от идеальной и поэтому среднеквадратичная ошибка получилась достаточно большой, что плохо. 

### Прогноз

Построим прогноз на 3 года

In [ ]:
start = df.index[-1] + pd.DateOffset(months=1)
end = start + pd.DateOffset(months=35)
start, end

In [ ]:
holt_winters_forecast = holt_winters.predict(start=start, end=end)

In [ ]:
plt.figure(figsize=(12,4), dpi=150)
df.plot(label="Values")
holt_winters_forecast.plot(label="Holt-Winters")
plt.legend(loc="upper left")
plt.show()

Прогноз получился достаточно плохим.

К сожалению, я не понял какой моделью стоит воспользоваться, так как у нас есть сезонность, но нет тренда.

- Модель Брауна работает в случае когда у нас нет ни тренда, ни сезонности.
- Модель Хольта работает когда у нас есть только трендовая составляющая.
- Модель Хольта-Уинтерса работает когда у нас есть и тренд, и сезонность.

**НО** даже если рассматривать что у нас есть тренд, то результаты получаются не лучше.

In [ ]:
holt_winters_trend = ExponentialSmoothing(df, initialization_method="estimated",
                                          trend="add", seasonal="add").fit()

In [ ]:
holt_winters_trend.model.params

In [ ]:
plt.figure(figsize=(12,4), dpi=150)
df.plot(label="CFE")
holt_winters_trend.fittedvalues.plot(label="Holt-Winters Trend")
plt.legend(loc="upper left")
plt.show()

**Среднеквадратичная ошибка**

In [ ]:
mse(df, holt_winters_trend.fittedvalues)